In [1]:
# %%capture
# !python -m pip install -U jinja2
# !python -m pip install -U psycopg2-binary

# !python -m pip freeze > requirements.txt

In [2]:
import subprocess

def get_latest_commit():

    repo_url = "https://github.com/cincinnatilibrary/duplicate-item-barcodes"
    commit_url = "{repo_url}/commit/{commit_hash}"
    try:
        # Run the git command to get the latest commit hash
        commit_hash = subprocess.check_output(["git", "rev-parse", "HEAD"]).strip().decode()
        # return commit_hash
        return commit_url.format(repo_url=repo_url, commit_hash=commit_hash)
    except subprocess.CalledProcessError as e:
        # print("Error fetching the latest commit hash:", e)
        return repo_url
    
version = get_latest_commit()

In [3]:
import json
import csv
from datetime import datetime, timedelta
import os

from sierra_db import execute_query_yield_rows, get_cursor
from chpl_email import send_email

from jinja2 import Environment, FileSystemLoader

# Set up the Jinja2 environment and load the template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('template.html')

try:
    with open('config.json') as f:
        config = json.load(f)
        dsn = config['dsn']
except Exception as e:
    print(e)
    
directory = "./output"

# Check if the directory exists
if not os.path.exists(directory):
    # If the directory doesn't exist, create it
    os.makedirs(directory)
    
# Get the current date and format it as a string
date_str = datetime.now().strftime("%Y-%m-%d")

# Now you can use this directory to save your file
filename = f"duplicate-item-barcodes_{date_str}.csv"
filepath = os.path.join(directory, filename)

In [4]:
config.get('send_list')

['ray.voelker@chpl.org', 'ray.voelker@gmail.com']

In [5]:
# sql = """\
# select 
#     v.field_content as barcode,
#     json_agg(rm.record_type_code || rm.record_num || 'a') as item_record_id_array 
# from 
#     sierra_view.varfield v 
#     join sierra_view.record_metadata rm on (
#         rm.id = v.record_id
#         and rm.record_type_code = 'i'
#         and rm.campus_code = ''
#     )
# where 
#     v.varfield_type_code = 'b'     -- barcode
# group by 
#     barcode
# having
#     count(distinct rm.id) > 1
# """

sql = """\
with dupe_barcode as (
	select
		v.field_content as barcode,
		date(max(rm.creation_date_gmt)) as latest_create_date
	from 
		sierra_view.varfield as v
		join sierra_view.record_metadata rm  on (
			rm.id = v.record_id
	        and rm.record_type_code = 'i'
	        and rm.campus_code = ''
		)
	where 
		v.varfield_type_code = 'b'  -- barcode
	group by 
		v.field_content
	having
		count(v.field_content) > 1
)
select
	dense_rank() over (order by db.latest_create_date desc, db.barcode) as barcode_group_num,
	db.barcode,
	rmi.record_type_code || rmi.record_num || 'a' as item_record_num,
	ir.icode1,
	ir.itype_code_num,
	brp.best_title as title,
	date(rmi.creation_date_gmt) item_created,
	date(rmi.record_last_updated_gmt) as item_updated 
from
	dupe_barcode as db
	left outer join sierra_view.phrase_entry as pe on (
		pe.index_tag || pe.index_entry = 'b' || lower(db.barcode)
		or pe.index_tag || pe.index_entry = 'b' || db.barcode
	)
	left outer join sierra_view.item_record as ir on
		ir.record_id = pe.record_id
	left outer join sierra_view.record_metadata rmi on
		rmi.id = ir.record_id
	left outer join sierra_view.bib_record_item_record_link as brirl on
		brirl.item_record_id = ir.record_id
	left outer join sierra_view.bib_record_property as brp on
		brp.bib_record_id = brirl.bib_record_id
order by
	db.latest_create_date desc,
	db.barcode
"""

with get_cursor(dsn=dsn) as cursor:
    rows = execute_query_yield_rows(cursor, sql, None)
    columns = next(rows)

    with open(filepath, 'w') as f:
        writer = csv.writer(f)    
        writer.writerow(columns)
        
        # Initialize an empty list to store the data for the template
        data = []
        
        for i, row in enumerate(rows):
            writer.writerow(row)
            
            # Add this row's data to the list for the template
            data.append(dict(zip(columns, row)))
            
            if i % 100 == 0:
                print('.', end='')
        print(f'.done ({i+1})')

..done (100)


In [6]:
# Render the template with the data
html = template.render(results=data, version=version)

# Now you can use the HTML string however you like, e.g., write it to a file or send it in an email
with open('report.html', 'w') as f:
    f.write(html)

# html = template.render(results=data)

In [7]:
send_email(
    smtp_username=config['smtp_username'], 
    smtp_password=config['smtp_password'], 
    subject="Duplicate Barcode for Item Records Report", 
    message="See attached.",
    html=html,
    from_addr="ray.voelker@chpl.org", 
    to_addr=config['send_list'], 
    files=[filepath, 'report.html']
)